In [4]:
import tkinter as tk
from tkinter import messagebox, simpledialog



class _ShipmentNode:
    def __init__(self, status, time, location):
        self.status = status  
        self.time = time
        self.location = location
        self.next = None  

class _ShipmentHistory:
    def __init__(self):
        self.head = None  

    def add_status(self, status, time, location):
        new_node = _ShipmentNode(status, time, location)
        if not self.head:
            self.head = new_node
        else:
            temp = self.head
            while temp.next:
                temp = temp.next
            temp.next = new_node  

    def get_status(self):
        status_list = []
        current = self.head
        while current:
            status_list.append((current.time, current.status, current.location))
            current = current.next
        return status_list  

class _PackageNode:
    def __init__(self, package_id, status, time, location, priority):
        self.package_id = package_id
        self.priority = priority
        self.history = _ShipmentHistory()
        self.history.add_status(status, time, location)  
        self.next = None  

class _DeliveryPriorityQueue:
    def __init__(self):
        self.head = None  

    def add_package(self, package_id, status, time, location, priority):
        new_package = _PackageNode(package_id, status, time, location, priority)
        
        if self.head is None or self.head.priority > priority:
            new_package.next = self.head
            self.head = new_package
        else:
            current = self.head
            while current.next and current.next.priority < priority:
                current = current.next
            while current.next and current.next.priority == priority:
                current = current.next  
            new_package.next = current.next
            current.next = new_package
        
        return new_package  

    def remove_package(self, package_id):
        if self.head is None:
            return False  

        if self.head.package_id == package_id:
            self.head = self.head.next
            return True  

        current = self.head
        while current.next and current.next.package_id != package_id:
            current = current.next

        if current.next:  
            current.next = current.next.next
            return True  

        return False  

class PackageTrackingSystem:
    def __init__(self):
        self.delivery_queue = _DeliveryPriorityQueue()
        self.packages = {}  

    def add_package(self, package_id, status, time, location, priority):
        if package_id in self.packages:
            return None 
        package = self.delivery_queue.add_package(package_id, status, time, location, priority)
        self.packages[package_id] = package
        return package

    def update_status(self, package_id, new_status, time, location):
        package = self.packages.get(package_id)
        if package:
            package.history.add_status(new_status, time, location)
            return True
        return False

    def get_history(self, package_id):
        package = self.packages.get(package_id)
        if package:
            return package.history.get_status()
        return []  

    def remove_package(self, package_id):
        if package_id in self.packages:
            found = self.delivery_queue.remove_package(package_id)
            if found:
                del self.packages[package_id]
                return True
        return False

    def get_delivered_packages(self):
        delivered = []
        for package_id, package in self.packages.items():
            history = package.history.get_status()
            if history and history[-1][1] == "Delivered":
                delivered.append(package_id)
        return delivered

    def get_all_packages(self):
        package_info = []
        for package_id, package in self.packages.items():
            history = package.history.get_status()
            if history:
                last = history[-1]
                package_info.append((package_id, package.priority, last))
        return package_info



class PackageTrackingApp:
    def __init__(self, root):
        self.system = PackageTrackingSystem()
        self.root = root
        self.root.title("Package Tracking System")

        self.main_frame = tk.Frame(root, padx=20, pady=20)
        self.main_frame.pack()

        tk.Label(self.main_frame, text="Package Tracking System", font=("Arial", 16, "bold")).pack(pady=10)

        btns = [
            (" Add Package", self.add_package),
            (" Update Status", self.update_status),
            (" View Delivered", self.show_delivered),
            (" View All Packages", self.view_all_packages),
            (" View History", self.view_history),
            (" Remove Package", self.remove_package),
            ("Exit", root.quit)
        ]

        for txt, cmd in btns:
            tk.Button(self.main_frame, text=txt, width=30, pady=5, command=cmd).pack(pady=4)

    def add_package(self):
        try:
            package_id = simpledialog.askstring("Add Package", "Enter Package ID:")
            if not package_id or package_id in self.system.packages:
                messagebox.showerror("Error", "Invalid or duplicate Package ID.")
                return
            status = simpledialog.askstring("Status", "Enter Initial Status:")
            time = simpledialog.askstring("Time", "Enter Time (YYYY-MM-DD HH:MM):")
            location = simpledialog.askstring("Location", "Enter Location:")
            priority = int(simpledialog.askstring("Priority", "Enter Priority (lower = higher priority):"))

            self.system.add_package(package_id, status, time, location, priority)
            messagebox.showinfo("Success", f"Package {package_id} added successfully!")
        except Exception as e:
            messagebox.showerror("Error", str(e))

    def update_status(self):
        try:
            package_id = simpledialog.askstring("Update Status", "Enter Package ID:")
            new_status = simpledialog.askstring("Status", "Enter New Status:")
            time = simpledialog.askstring("Time", "Enter Time:")
            location = simpledialog.askstring("Location", "Enter Location:")
            updated = self.system.update_status(package_id, new_status, time, location)
            if updated:
                messagebox.showinfo("Updated", f"Status updated for package {package_id}.")
            else:
                messagebox.showerror("Error", "Package not found.")
        except Exception as e:
            messagebox.showerror("Error", str(e))

    def show_delivered(self):
        delivered = self.system.get_delivered_packages()
        if delivered:
            messagebox.showinfo("Delivered Packages", "\n".join(delivered))
        else:
            messagebox.showinfo("Delivered Packages", "No packages have been delivered yet.")

    def view_all_packages(self):
        package_info = self.system.get_all_packages()
        if package_info:
            display_text = ""
            for pkg_id, priority, (time, status, location) in package_info:
                display_text += f"ID: {pkg_id}, Priority: {priority}, Last: [{time} | {status} | {location}]\n"
            messagebox.showinfo("All Packages", display_text)
        else:
            messagebox.showinfo("All Packages", "No packages in the system.")

    def view_history(self):
        package_id = simpledialog.askstring("View History", "Enter Package ID:")
        history = self.system.get_history(package_id)
        if history:
            hist_str = "\n".join([f"{t} - {s} - {l}" for t, s, l in history])
            messagebox.showinfo(f"History for {package_id}", hist_str)
        else:
            messagebox.showinfo("History", "No history found or invalid package ID.")

    def remove_package(self):
        package_id = simpledialog.askstring("Remove Package", "Enter Package ID to remove:")
        removed = self.system.remove_package(package_id)
        if removed:
            messagebox.showinfo("Removed", f"Package {package_id} removed successfully.")
        else:
            messagebox.showerror("Error", "Package not found or already removed.")



if __name__ == "__main__":
    root = tk.Tk()
    app = PackageTrackingApp(root)
    root.mainloop()
